In [ ]:
What is Dependency Injection and Why Use Depends?

Dependency Injection: A design pattern where a piece of code (like a FastAPI path operation function) receives the objects it needs to operate, rather than creating them itself.

The Role of Depends: It's FastAPI's way of saying, "Hey, my path operation function needs this thing. Please figure out how to get it and provide it to me."

Benefits of using Depends:

Modular Code: Breaks your code into smaller, reusable units (the dependencies). This makes your code more organized and testable.
Reduced Coupling: Your path operation functions don't know the details of how their dependencies are created. This keeps things flexible.
Shared Logic: You can reuse the same dependency logic across multiple path operations.
Enhanced Security and Authentication: Depends helps to inject authentication mechanisms and ensure user permissions systematically.
Automatic Documentation: Dependencies are integrated into the OpenAPI (Swagger) documentation of your API.
How to Use Depends

1. Define a Dependency Function

Python
from fastapi import Depends

def get_db():
    db_session = some_database_logic()
    try:
        yield db_session
    finally:
        db_session.close()

Use code with caution.
2. Use Depends in a Path Operation

Python
from fastapi import APIRouter, Depends

router = APIRouter()

@router.get("/items/")
async def get_items(db=Depends(get_db)):
    return db.query("SELECT * FROM items").all()
Use code with caution.
Explanation

get_db is our dependency function. It could handle database connections, authentication, or external service interactions.
In our path operation function get_items, we declare db=Depends(get_db). FastAPI will see this and do the following:
When a request comes to get_items:
FastAPI executes get_db.
The result of get_db (e.g., the database session) is provided as the value of the db parameter in your path operation function.
Advanced Depends Features

Sub-dependencies: Dependencies can call other dependencies. This lets you build complex relationships between required objects.
Classes as Dependencies: You can declare a class as a dependency; FastAPI will create an instance and inject it.
Parameterization: You can create dependencies that take parameters, providing customization and fine-grained control.
Key Points

Return Value: The value returned by a dependency function is what gets injected into your path operation function.
Caching: FastAPI may cache the return value of dependency functions within a single request to reduce redundant calculations.
Yield: Using yield in dependencies is often crucial for resource management (like closing database connections).
Let me know if you want specific examples of sub-dependencies, classes as dependencies, parameterization, or a more complex use-case. I'm ready to dive deeper!

profile picture

##################################################################################################
more examples 

1
Absolutely! Here's a breakdown of some more advanced examples demonstrating the flexibility of FastAPI's Depends:

1. Sub-Dependencies

Python
from fastapi import Depends, HTTPException, status

async def get_db():  # Simulating a database connection
    db_session = {"connection": "established"}
    yield db_session

async def get_current_user_from_db(db: dict = Depends(get_db)):
    if not db:
        raise HTTPException(status_code=status.HTTP_503_SERVICE_UNAVAILABLE)  # Simulating DB error
    return {"username": "user123"}

async def read_items(user: dict = Depends(get_current_user_from_db)):
    return {"items": ["item 1", "item 2"], "user": user}
Use code with caution.
get_current_user_from_db depends on get_db. FastAPI will resolve this chain by first getting a database session from get_db.
This allows you to layer dependencies cleanly. Authentication often relies on database access, demonstrating this hierarchical structure.
2. Classes as Dependencies

Python
from fastapi import Depends, Security
from pydantic import BaseModel

class CommonQueryParams:
    def __init__(self, q: str = None, skip: int = 0, limit: int = 100):
        self.q = q
        self.skip = skip
        self.limit = limit

async def get_query_params(commons: CommonQueryParams = Depends()):
    return commons
Use code with caution.
The CommonQueryParams class acts as a dependency. Inside path operations, you can now directly use it for common query parameters.
This centralizes your validation and keeps code DRY.
3. Parameterization

Python
from fastapi import Depends

def get_logged_in_user(login_token: str = Depends()):
    # Logic to decode, validate login_token
    return {"username": "current_user"}  

async def read_private_data(user: dict = Depends(get_logged_in_user)):
    # Access private data with current user
    return  {"message": "Secret info!"}
Use code with caution.
The get_logged_in_user dependency can be made flexible by accepting parameters like the login_token. This allows you to modify its behavior depending on inputs, potentially validating different login token formats.
Complex Use Case: Authentication + Authorization

Python
from fastapi import Depends, Security, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm

fake_users_db = {"admin": {"password": "SuperSecretPassword"}}

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")  # Simulating OAuth2 flow

# ... (get_db dependency etc.)

async def get_current_user(token: str = Depends(oauth2_scheme)):
    # Decode token, verify user, this would usually involve DB interaction
    return fake_users_db.get("admin")  

async def get_admin_user(current_user: dict = Security(get_current_user, scopes=["admin"])):
    if current_user.get("username") != "admin":
        raise HTTPException(status_code=status.HTTP_403_FORBIDDEN)
    return current_user

async def admin_endpoint(admin: dict = Depends(get_admin_user)):
    return {"message": "Welcome Admin!"}
Use code with caution.
We combine token-based authentication (oauth2_scheme) with granular authorization (scopes).
Note the use of Security to specify required scopes for the admin dependency.